# Tests MSSQL Python

Frist create a .env file:

```env
MSSQL_SERVER="..."
MSSQL_DATABASE="..."
MSSQL_UID="..."
MSSQL_PWD="..."
```

In [1]:
%reload_ext dotenv
%dotenv

import os

config = {
    "server": os.environ["MSSQL_SERVER"],
    "database": os.environ["MSSQL_DATABASE"],
    "uid": os.environ["MSSQL_UID"],
    "pwd": os.environ["MSSQL_PWD"],
}

## PyODBC

**Microsoft ODBC Driver for SQL Server is required before to install pyodbc**.

Please follow the instructions to install the driver for your OS. See the [docs](https://learn.microsoft.com/en-us/sql/connect/odbc/microsoft-odbc-driver-for-sql-server) for details.

To install run this command:
```bash
poetry add pyodbc
```

In [ ]:
import pyodbc

sql_server_drivers = list(filter(lambda x: "SQL Server" in x, pyodbc.drivers()))
conn_str = (
    f"DRIVER={{{sql_server_drivers[0]}}};"
    f'SERVER=tcp:{config["server"]};PORT=1433;'
    f'DATABASE={config["database"]};'
    f'UID={config["uid"]};'
    f'PWD={config["pwd"]}'
)
display(conn_str)

with pyodbc.connect(conn_str) as conn:
    with conn.cursor() as cursor:
        rows = cursor.execute("SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE';")
        # for row in rows:
        #     display(row[2])
        for column in rows.columns():
            display(column)

## PyTDS

To install run this command:

```bash
poetry add python-tds
```

If you want to use TLS you should also install pyOpenSSL package:

```bash
poetry add pyOpenSSL
```

For a better performance install bitarray package too:

```bash
poetry add bitarray
```

To use Kerberos on non-Windows platforms (experimental) install kerberos package:

```bash
poetry add kerberos
```

In [ ]:
import pytds

with pytds.connect(dsn=config["server"], database=config["database"], user=config["uid"], password=config["pwd"], cafile="../test.cer") as conn:
    with conn.cursor() as cursor:
        cursor.execute("select 1")

## MiniAB

**miniab** requires duckdb and pyarrow and they should be installed before to use it.

To install duckdb and pyarrow run this command:

```bash
poetry add duckdb pyarrow
```

In [ ]:
import duckdb
import miniab as ab

source = ab.get_source(
    "source-mssql",
    config = config,
    sync = True
)
source.select_all_streams()
source.check()

cache = ab.get_default_cache()
result = source.read(cache=cache)

tteam_tbl = result.to_arrow("TTeam")
duckdb.sql("SELECT * FROM tteam_tbl LIMIT 10;").show()

Sync Progress: SnowflakeSource -> DuckdbCache
Started reading from source at 10:41:20
 * Received records for 198 streams:
  - 11 Raw_Arpeges_SKU_Temp                  
  - 158 TTeam                  
  - 2,370 TSector                  
  - 335 R_CutSales_ByMonth                  
  - 4,030 Raw_Arpeges_SKU                  
  - 2,312 TTeamProduct                  
